In [185]:
#!python 3.9.7

from skimage import io
import tkinter as tk
from tkinter import filedialog, messagebox
import matplotlib.pyplot as plt
from os import listdir, _exit
from os.path import isfile, join
import logging, sys, time
logging.basicConfig(stream=sys.stderr, level=logging.INFO)

%matplotlib qt


In [186]:
MANUALMODE = False

if MANUALMODE:
    root = tk.Tk()
    root.wm_attributes('-topmost', True)
    root.withdraw()
    manualpath = filedialog.askopenfilename(filetypes=[('Tiff-Files','*.tif'),('ALL files','*')])

In [187]:
if MANUALMODE: 
    image = io.imread(manualpath)
    plt.subplots(figsize=(6,6))
    plt.imshow(image[0,:,:])
    plt.show()

In [188]:
# Create Tk root
#root = tk.Tk()
# Hide the main window
#root.withdraw()
#root.call('wm', 'attributes', '.', '-topmost', True)
#filedialog.askdirectory(initialdir='./')
#window.mainloop()
#window.wm_withdraw()

In [189]:
# -*- coding: utf-8 -*-
"""
Created on Tue Nov  1 16:09:06 2022

@author: damlatetiker
"""
import numpy as np
from skimage import filters, morphology, measure
import copy
import matplotlib.pyplot as plt
import pandas as pd
from skimage.registration import phase_cross_correlation
from scipy.ndimage import fourier_shift
import cv2

def detect_frap(curr_img):
    # this function determines the last frame before and first frame after bleaching by:
    # defining a rough estimate for the bleached condenste via thresholding (li's method is used) the first frame
    # summing the intensity over all pixels in this ROI over the whole time series
    # determining the sudden drop in the intensity
    # curr_img should be an array of M x N x K with M indexing the time
    
    # Get the first frame from the image stack and determine the ideal threshold with li's method
    first_frame = curr_img[0].astype(float)
    thresh_li = filters.threshold_li(first_frame)
    
    # Create a mask where foreground is True and background is False by thresholding
    mask = first_frame > thresh_li
    # Remove objects smaller than a given size (for now 20 pixels) to get rid of other structures than the desired condenste
    mask = morphology.remove_small_objects(mask, 15)
    
    # initialize a list that would hold the sum of pixel intensities within the ROI over the time series
    int_seq = []
    for i in range(len(curr_img)):
        int_seq.append(np.sum(np.where(mask==True, curr_img[i], 0)))
    # Change the datatype of the intensity sequence to float, otherwise there are problems
    int_seq = np.array(int_seq).astype(float)
    # Take the 'derivative' of the intensity sequence, it should be 0 eg. constant function except for one very small number, the drop of intensity
    der = np.diff(int_seq)
    # Find where the derivative has a minimum, this is the index of the frame before bleaching
    ind_before = np.argmin(der)
    ind_after = ind_before+1
    
    return ind_before, ind_after


def correct_drift(movie,alignment_slice=None,upsample_factor=4, verbose = 0):

    # Define the sparse sample parameters
    feature_params = dict(maxCorners=50, qualityLevel=0.001, minDistance=5)


    # Define the optical flow parameters
    lk_params = dict(winSize=(30, 30),\
                    maxLevel=4,\
                    criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03),\
                    flags=cv2.OPTFLOW_LK_GET_MIN_EIGENVALS,\
                    minEigThreshold=1e-4)

    # Define the initial parameters
    old_frame = movie[0]
    old_gray = np.uint8(old_frame)
    h, w = old_gray.shape
    shift_x = np.zeros((len(movie),))
    shift_y = np.zeros((len(movie),))    

    # Initialize the corrected_shift_movie array
    corrected_shift_movie = np.zeros_like(movie)

    # Iterate through each frame in the movie
    for i in range(1, len(movie)):
        # Get the next frame and convert to grayscale
        frame = movie[i]
        frame_gray = np.uint8(frame) #cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        # Calculate the optical flow using sparse Lucas-Kanade method
        p0 = cv2.goodFeaturesToTrack(old_gray,  mask=None, **feature_params)
        p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)
        
        # Calculate the mean shift
        good_new = p1[st==1]
        good_old = p0[st==1]

        M, _ = cv2.estimateAffinePartial2D(good_old, good_new)

        # Warp current frame based on optical flow
        corrected_shift_movie[i] = cv2.warpAffine(frame, np.float32(M), (frame.shape[1], frame.shape[0]))
            
        # Set the current frame as the previous frame for the next iteration
        old_gray = frame_gray.copy()

    # Set the first frame in the corrected_shift_movie to be the same as in the original movie
    corrected_shift_movie[0] = movie[0]
    return corrected_shift_movie


def refine_ROIs(curr_img, ind_before, ind_after):
    # Refine the ROI by taking the average of all frames up to bleaching and thresholding this average image
    # one needs to take ind_before+1 since indexing starts at 0
    frames_before = curr_img[:ind_before]
    img_avg = np.mean(frames_before, axis=0)
    
    # Determine the ideal threhold for the average image with li's method and apply it to get a mask
    thresh_li1 = filters.threshold_li(img_avg)
    mask_roi1 = img_avg>thresh_li1
    mask_roi1 = morphology.remove_small_objects(mask_roi1, 20)
    
    # get the first frame after bleaching
    first_after = curr_img[ind_after].astype(float)
    
    # Determine the ideal threhold for the first frame after bleaching with li's method and apply it to get a mask
    thresh_li2 = filters.threshold_li(first_after)
    mask_int = first_after>thresh_li2
    mask_int = morphology.remove_small_objects(mask_int, 20)
    
    # The second ROI (bleaching site) are the pixels that are true in the first mask and false in the second one
    mask_roi2 = copy.deepcopy(mask_roi1)
    mask_roi2[mask_int==True] = False
    
    # Make sure you have only one region, as the process above may introduce some problems
    label_image,label_num = measure.label(mask_roi2, return_num = True)
    if label_num > 1:

        assert( label_image.max() != 0 ) # assume at least 1 CC
        mask_roi2 = label_image == np.argmax(np.bincount(label_image.flat)[1:])+1

        #mask_roi2 = mask_roi2()
        #raise NotImplementedError('The mask for the bleached area should have 1 label but instead has {}'.format(np.amax(label_image)))
        # It is not yet implemented how to deal with such an incorrect segmentation
    elif label_num == 0 :
        mask_roi2 = copy.deepcopy(mask_roi1)
    return mask_roi1, mask_roi2

def refine_ROIs2(curr_img, ind_before, ind_after):
    print('The second method for ROI calculation is not implemented yet')
    return 0, 0
def get_background_(mic_img, area_cond):
    avg_img = np.mean(mic_img, axis=0)
    thresh_mean = filters.threshold_mean(avg_img)
    backg_mask = avg_img<thresh_mean
    backg_size = int(np.sqrt(3*area_cond))
    
    return

def export_intensity(curr_img, mask_roi1, mask_roi2, file_path):
    # Total area
    img_area_1 = copy.deepcopy(curr_img)
    img_area_1[:, mask_roi1==False] = 0
    area_total_ = np.sum(mask_roi1)
    area_total = np.ones(len(curr_img))*area_total_
    mean_intensity_whole_area = np.sum(np.sum(img_area_1, axis=1), axis=1)/area_total_
    
    # Bleached area
    img_area_2 = copy.deepcopy(curr_img)
    img_area_2[:, mask_roi2==False] = 0
    area_bleached_ = np.sum(mask_roi2)
    area_bleached = np.ones(len(curr_img))*area_bleached_
    mean_intensity_bleached_area = np.sum(np.sum(img_area_2, axis=1), axis=1)/area_bleached_
    
    # Unbleached area
    mask_roi3 = copy.deepcopy(mask_roi1)
    mask_roi3[mask_roi2==True] = False
    img_area_3 = copy.deepcopy(curr_img)
    img_area_3[:, mask_roi3==False] = 0
    area_unbleached_ = np.sum(mask_roi3)
    area_unbleached = np.ones(len(curr_img))*area_unbleached_
    mean_intensity_unbleached_area = np.sum(np.sum(img_area_3, axis=1), axis=1)/area_unbleached_
    
    ## TODO: Parse frame times and write cumulative frame time
    
    ## TODO: Implement prebleach series length
    
    # doubleNoormlization = [(TotalROI_1-20 - Bg_1-20) * (TotalROI_slice - Bg_slice)] / [(Bleached_1-20 - Bg_1-20) * (Bleached_slice - Bg_slice)] 
    # singleNormalization = [Bleached_slice - Bg_slice] / [Bleach_1-20 - Bg_1-20]
    
    #TODO New script: concatenate double and single norm from whole FRAP experiment, first colum frame time
    
    results = {'Area(all)':area_total,
               'Mean_intensity(all)': mean_intensity_whole_area,
               'Area(bleached)': area_bleached,
               'Mean_intensity(bleached)': mean_intensity_bleached_area,
               'Area(unbleached)': area_unbleached,
               'Mean_intensity(unbleached)': mean_intensity_unbleached_area}
    
    results = pd.DataFrame(data=results)
    excel_path = file_path[:-4]+'.xlsx'
    print('wrote results to' + excel_path)
    results.to_excel(excel_path)   
    return
    

In [190]:
# -*- coding: utf-8 -*-
"""
Created on Tue Nov  1 11:10:18 2022

@author: damlatetiker
"""
import numpy as np
import matplotlib.pyplot as plt
from skimage import io, measure, filters
from matplotlib.widgets import Slider, RectangleSelector, Button, PolygonSelector
import tkinter as tk
from tkinter import filedialog, messagebox
from os import listdir, _exit
from os.path import isfile, join
import logging, sys, time
import cv2
logging.basicConfig(stream=sys.stderr, level=logging.INFO)

class ImageVisualizer:
    def __init__(self):
        return
    def __init__(self, path):

        self.file_path = path   
        self.supersampling = 2
        self.drift_correction = True

        self.createFigure(self.file_path)
        self.update()
        self.connect()
        return

    # def __init__(self, path, filelist= []):
    #     if filelist.len() == 0 and path.endswith('.tif'):
    #         self.file_path = path
    #     elif filelist.len() > 0:
    #         self.file_dir = path
    #         self.file_list = filelist
    #         self.file_path = self.file_dir + '/' + self.file_list.pop(0)
    #     else:
    #         logging.debug(path)
    #         raise Exception('Unknown file path')

    #     self.createFigure()
    #     self.update()
    #     self.connect()
    #     return


    def createFigure(self, path):
        self.fig, self.ax = plt.subplots(figsize=(9,6))      
        self.mic_img = io.imread(path)


        self.slices, rows, cols = self.mic_img.shape

        self.mic_img_dim = (int(cols * self.supersampling), int(rows * self.supersampling))
        if self.supersampling > 1:
            print('Supersampling image to a final resolution of {}x{}'.format(self.mic_img_dim[0], self.mic_img_dim[1]))
            self.mic_img_supersampled_tmp = np.zeros((self.slices,self.mic_img_dim[0], self.mic_img_dim[1]))
            for i in range(self.slices):
                self.mic_img_supersampled_tmp[i,:,:] = cv2.resize(self.mic_img[i,:,:],(self.mic_img_dim[0], self.mic_img_dim[1]), interpolation = cv2.INTER_LANCZOS4) 
            self.mic_img = self.mic_img_supersampled_tmp
            del self.mic_img_supersampled_tmp

        axslid = self.fig.add_axes([0.3, 0.03, 0.4, 0.03])
        self.slider = Slider(ax=axslid, label='Slice', valmin=0, valmax=self.mic_img.shape[0]-1, valinit=0, valstep=1, initcolor='none', color='lightgrey')
        

        self.ax.set_aspect('equal')
        self.ax.set_title('use scroll wheel to navigate images')
        self.selector = RectangleSelector(self.ax,self.select_roi)
        
        self.curr_img = self.mic_img
        self.ind = 0

        self.im = self.ax.imshow(self.mic_img[self.ind, :, :], cmap='gray')#, vmax=2000)
        self.curr_img_bbox = [-0.5, self.mic_img_dim[1]+.5, self.mic_img_dim[0]+.5, -0.5]

        self.update()
        return
        
# #def load_next_stack(self, event):    
#         logging.info(self.file_list)       
#         if self.file_list.len() > 0:              
#             self.file_path = self.file_dir + '/' + self.file_list.pop(0)  
#             plt.close('all')
#             #self.createFigure(self.file_path)            
#             return
#         else:
#             messagebox.showinfo('Finished!')
#             plt.close()
#             _exit() 
    
    def connect(self):
        self.fig.canvas.mpl_connect('scroll_event', self.on_scroll)
        self.fig.canvas.mpl_connect('key_press_event', self.on_press)
        self.slider.on_changed(self.update_from_slider)
        self.fig.canvas.mpl_connect('close_event',lambda event: self.fig.canvas.stop_event_loop())
        return
    
    def connect2(self):
        self.but1.on_clicked(self.ROI_method1)
        self.but2.on_clicked(self.ROI_method2)
        self.but3.on_clicked(self.export)
        #self.but4.on_clicked(self.load_next_stack)
        return
    
    def on_scroll(self, event):
        #print("%s %s" % (event.button, event.step))
        if event.button == 'up':
            self.ind = (self.ind + 1) % self.slices
        else:
            self.ind = (self.ind - 1) % self.slices
        self.slider.set_val(self.ind)
        self.update()
        return
        
    def on_press(self, event):
        if event.key == 'up' or event.key == 'right':
            self.ind = (self.ind + 1) % self.slices
        elif event.key == 'down'or event.key == 'left':
            self.ind = (self.ind - 1) % self.slices
        elif event.key == 'shift+alt':
            self.curr_img = self.mic_img
            self.curr_img_bbox = [-0.5, self.mic_img_dim[1] + .5, self.mic_img_dim[0] + .5, -0.5]
            self.ax.set_title('use scroll wheel to navigate images')
            for line in self.ax.get_lines(): # ax.lines:
                line.remove()
            self.selector.set_active(True)
        elif event.key == 'enter':
            self.ind_before, self.ind_after = detect_frap(self.curr_img)    
            if self.drift_correction:            
                print('Starting drift correction...')
                self.curr_img = correct_drift(self.curr_img, self.ind_after)      
            self.ax.set_title('Now showing the ROIs calculated with method 1.\n'+
                    'To swith press button ROI method 2.\nTo see the background ROI press shift alt (not implemented yet)\n'+
                    'To export press button Export\nTo export and load the next stack, press Next Stack\n')
            
            #tool_bar = self.fig.canvas.manager.toolbar
            #tool_bar.mode = ''
            self.selector.set_active(False)
            ax_met1 = self.fig.add_axes([0.8, 0.8, 0.1, 0.06])
            ax_met2 = self.fig.add_axes([0.8, 0.7, 0.1, 0.06])
            ax_met3 = self.fig.add_axes([0.8, 0.6, 0.1, 0.06])            
            #ax_met4 = self.fig.add_axes([0.8, 0.5, 0.1, 0.06])
            self.but1 = Button(ax_met1, 'ROI method 1')
            self.but2 = Button(ax_met2, 'ROI method 2 \n not implem. ')
            self.but3 = Button(ax_met3, 'Export')
            #self.but4 = Button(ax_met4, 'Next Stack')
            # mask_roi1 is the total area, mask_roi2 is the bleached area
            # To get the unbleached area (necessary for exporting, see the function export_intensity) we subtract the second mask from the first one
            self.mask_roi1, self.mask_roi2 = refine_ROIs(self.curr_img, self.ind_before, self.ind_after)
            self.area_cond = np.sum(self.mask_roi1)
            self.get_background()
            self.draw_ROI(self.mask_roi1, self.mask_roi2)
            self.connect2()
        self.slider.set_val(self.ind)
        self.update()
        return
  
    
    def ROI_method1(self, event):
        self.ax.set_title('Now showing the ROIs calculated with method 1.\nTo swith press button ROI method 2.\nTo see the background ROI press shift alt (not implemented yet)\nTo export press button Export')
        self.mask_roi1, self.mask_roi2 = refine_ROIs(self.curr_img, self.ind_before, self.ind_after)
        self.draw_ROI(self.mask_roi1, self.mask_roi2)
        return
    
    def ROI_method2(self, event):
        self.ax.set_title('Now showing the ROIs calculated with method 2.\nTo swith press button ROI method 1.\nTo see the background ROI press shift alt (not implemented yet)\nTo export press button Export')
        self.mask_roi1, self.mask_roi2 = refine_ROIs2(self.curr_img, self.ind_before, self.ind_after)
        self.draw_ROI(self.mask_roi1, self.mask_roi2)
        return
    
    def get_background(self):
        get_background_(self.mic_img, self.area_cond)
        return
        
    def export(self, event):
        export_intensity(self.curr_img, self.mask_roi1, self.mask_roi2, self.file_path)
        self.Finished = True
        return
    
    def draw_ROI(self, mask1, mask2):
        #for line in self.ax.get_lines(): # ax.lines:
        #    line.remove()
        contour1 = measure.find_contours(mask1 == 1)[0]
        contour2 = measure.find_contours(mask2 == 1)[0]
        bbox = self.curr_img_bbox
        poly_full_props = dict(color='r', linestyle='--', linewidth=2, alpha=0.5)           
        self.poly_full = PolygonSelector(self.ax, onselect=self.on_select_poly1, props=poly_full_props, grab_range=10)
        # Add three vertices
        poly_full_vertices = []
        for i in range(1,len(contour1[:,1]),4):
            poly_full_vertices.append((contour1[i,1]+bbox[0]+0.5, contour1[i,0]+bbox[3]+0.5))
        self.poly_full.verts = poly_full_vertices

        poly_bleach_props = dict(color='b', linestyle='--', linewidth=2, alpha=0.5)           
        self.poly_bleach = PolygonSelector(self.ax, onselect=self.on_select_poly2, props=poly_bleach_props, grab_range=10)
        # Add three vertices
        poly_bleach_vertices = []
        for i in range(1,len(contour2[:,1]),4):
            poly_bleach_vertices.append((contour2[i,1]+bbox[0]+0.5, contour2[i,0]+bbox[3]+0.5))
        self.poly_bleach.verts = poly_bleach_vertices

        #self.ax.plot(contour1[:,1]+bbox[0]+0.5, contour1[:,0]+bbox[3]+0.5, c='r')
        #self.ax.plot(contour2[:,1]+bbox[0]+0.5, contour2[:,0]+bbox[3]+0.5, c='b')
        self.im.set_extent(bbox)
        return
    
    def update(self):
        self.im.set_extent(self.curr_img_bbox)
        self.im.set_data(self.curr_img[self.ind, :, :])
        self.ax.set_aspect('equal')
        self.ax.set_ylabel('slice %s' % self.ind)
        self.im.axes.figure.canvas.draw()
        self.fig.canvas.draw_idle()
        return

    def update_from_slider(self, val):
        self.ind = val
        self.update()
        return
    
    def select_roi(self, eclick, erelease):
        
        self.curr_img = self.mic_img[:, int(eclick.ydata):int(erelease.ydata), int(eclick.xdata):int(erelease.xdata)]
        self.curr_img_bbox = [int(eclick.xdata)-0.5, int(erelease.xdata)-0.5, int(erelease.ydata)-0.5, int(eclick.ydata)-0.5]
        self.ax.set_title('Are you happy with the current ROI?\nTo reset press shift and alt\nTo continue with automated selection press enter')
        self.update()
        return
    
    def on_select_poly1(self, vertices):
        #self.
        #self.poly1 = plt.Polygon(vertices, animated=True, color='r')
        #self.ax.add_patch(self.poly1)
        #print('Vertices: {}'.format(vertices))
        #self.poly1.verts = vertices
        #self.poly1.update()
        #self.im.axes.figure.canvas.draw()
        self.redraw_roi1(vertices)
        self.fig.canvas.draw_idle() 
        
    def on_select_poly2(self, vertices):
        #self.
        #self.poly1 = plt.Polygon(vertices, animated=True, color='r')
        #self.ax.add_patch(self.poly1)
        #print('Vertices: {}'.format(vertices))
        #self.poly1.verts = vertices
        #self.poly1.update()
        #self.im.axes.figure.canvas.draw()
        self.redraw_roi2(vertices)
        self.fig.canvas.draw_idle()
    
    def redraw_roi1(self, vertices):
        black_image = np.zeros((self.mask_roi1.shape[0],self.mask_roi1.shape[1]),dtype=np.uint8)
        vertices_normalized = np.array([*vertices])
        vertices_normalized[:,0] = vertices_normalized[:,0] - self.curr_img_bbox[0] -0.5
        vertices_normalized[:,1] = vertices_normalized[:,1] - self.curr_img_bbox[3] -0.5
        self.mask_roi1= cv2.fillPoly(black_image, pts =np.int32([vertices_normalized]), color=(255,255,255))
        #self.mask_roi1 = cv2.threshold(self.mask_roi1, 128, 255, cv2.THRESH_BINARY)[1]
        self.mask_roi1 = self.mask_roi1 > 128
        #self.ax.imshow(self.mask_roi1, cmap='gray')#, vmax=2000)
        #self.mask_roi1 = cv2.threshold(self.mask_roi1,125)
        #self.mask_roi1[self.mask_roi1 ==(255,255,255)] = True 

    def redraw_roi2(self, vertices):
        black_image = np.zeros((self.mask_roi2.shape[0],self.mask_roi2.shape[1]),dtype=np.uint8)
        vertices_normalized = np.array([*vertices])
        vertices_normalized[:,0] = vertices_normalized[:,0] - self.curr_img_bbox[0] -0.5
        vertices_normalized[:,1] = vertices_normalized[:,1] - self.curr_img_bbox[3] -0.5
        self.mask_roi2 = cv2.fillPoly(black_image, pts =np.int32([vertices_normalized]), color=(255,255,255))
        #self.mask_roi2 = cv2.threshold(self.mask_roi2, 128, 255, cv2.THRESH_BINARY)[1]
        self.mask_roi2 = self.mask_roi2 > 128
        self.ax.imshow(self.mask_roi2, cmap='gray')#, vmax=2000)
        #self.mask_roi1 = cv2.threshold(self.mask_roi1,125)
        #self.mask_roi1[self.mask_roi1 ==(255,255,255)] = True

    def get_canvas(self):
        return self.fig.canvas
     

In [191]:
from threading import Thread
from pprint import pprint

def checkFileListOverwrite(file_dir):
    file_list = [f for f in listdir(file_dir) if isfile(join(file_dir, f)) and f.endswith('.tif')]
    xlsx_files = [f.removesuffix('.xlsx') for f in listdir(file_dir) if isfile(join(file_dir, f)) and f.endswith('.xlsx')]
    result = messagebox.askyesnocancel('Old processing files found!','Would you like to overwrite them?')
    if result:  
        return file_list
    elif result == False:
        return [t for t in file_list if t.removesuffix('.tif') not in xlsx_files]
    else:
        return None

# Create Tk root
tk_root = tk.Tk()
# Hide the main window
tk_root.withdraw()
tk_root.call('wm', 'attributes', '.', '-topmost', True)
#file_dir = filedialog.askdirectory(initialdir='./')
file_dir = r'C:\Users\Johann\Documents\Repositories\FRAP-Processing\raw'#FRAP GFP-Ede1-GFP'
file_list = checkFileListOverwrite(file_dir)
pprint(file_list)

['stk_0001_20221012_FRAP_GFP-Ede1-GFP_empty.lif - FRAP 003_FRAP Pre '
 'Series07.tif',
 'stk_0002_20221012_FRAP_GFP-Ede1-GFP_empty.lif - FRAP 003 FRAP Pre '
 'Series07.tif',
 'stk_0002_20221012_FRAP_GFP-Ede1-GFP_empty.lif - FRAP 003_FRAP Pre '
 'Series07.tif',
 'stk_0003_20221012_FRAP_GFP-Ede1-GFP_empty.lif - FRAP 003_FRAP Pre '
 'Series07.tif',
 'stk_0004_20221012_FRAP_GFP-Ede1-GFP_empty.lif - FRAP 003_FRAP Pre '
 'Series07.tif',
 'stk_0005_20221012_FRAP_GFP-Ede1-GFP_empty.lif - FRAP 003_FRAP Pre '
 'Series07.tif',
 'stk_0006_20221012_FRAP_GFP-Ede1-GFP_empty.lif - FRAP 003_FRAP Pre '
 'Series07.tif',
 'stk_0007_20221012_FRAP_GFP-Ede1-GFP_empty.lif - FRAP 003_FRAP Pre '
 'Series07.tif',
 'stk_0008_20221012_FRAP_GFP-Ede1-GFP_empty.lif - FRAP 003_FRAP Pre '
 'Series07.tif',
 'stk_0009_20221012_FRAP_GFP-Ede1-GFP_empty.lif - FRAP 003_FRAP Pre '
 'Series07.tif']


In [192]:
for f in file_list:
    file_path = file_dir + '/' + f
    print(file_path)
    try:
        vis = ImageVisualizer(file_path)
        #plt.show(block=True)
        #t1 = Thread(target = vis.get_canvas().start_event_loop(), name='ImageVisualizer')
        #t1.daemon = True
        #t1.start()        
    except Exception as e:
        logging.info('error during execution of ImageVisualizer')
        logging.info('{}'.format(str(e)))
        sys.exit()
    if not messagebox.askokcancel('Do you want to continue?','Do you want to continue?') :
        sys.exit()

C:\Users\Johann\Documents\Repositories\FRAP-Processing\raw/stk_0001_20221012_FRAP_GFP-Ede1-GFP_empty.lif - FRAP 003_FRAP Pre Series07.tif
Supersampling image to a final resolution of 512x512
C:\Users\Johann\Documents\Repositories\FRAP-Processing\raw/stk_0002_20221012_FRAP_GFP-Ede1-GFP_empty.lif - FRAP 003 FRAP Pre Series07.tif
Supersampling image to a final resolution of 512x512
Starting drift correction...


SystemExit: 